<a href="https://colab.research.google.com/github/Darally06/Datos-2---Lab2/blob/main/LabGrafos_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MANEJO AEROPUERTOS**
Distancias de vuelos

*Imports y requerimientos*

In [5]:
import folium
import pandas as pd
import itertools
import heapq
from collections import deque
from math import radians, sin, cos, sqrt, asin
from IPython.display import display

*Clase para manejar el sistema*

In [6]:
class MapHandler:

    def __init__(self):
        """
        Incializa la instancia
        Carga de datos necesarios, inicia procesos para el manejo de datos
        """
        self.mapa = folium.Map(zoom_start=4)                # Mapa incial
        self.ubicaciones_agregadas = set()                  # Conjunto control de ubicaciones agregadas
        self.df = pd.read_csv("/content/flights_final.csv") # Carga de archivo

        # Extracción de información, codigos únicos, construcción del grafo
        self.airport_info = self.extract()
        self.airport_codes = self.unique_code()
        self.distance_matrix= self.matriz_distance()
        self.grafo = self.grafo_built()

        # Agregar todos los aereopuertos al mapa
        # self.agregate_airports()
        # display(self.mapa)

    def grafo_built(self):
        """
        Construir el grafo de conexiones. Utiliza los datos del DF para crear las conexiones y distancias entre ellos.
        """
        grafo = {}                                            # Diccionario del grafo

        for index, row in self.df.iterrows():                 # Recorre cada fila del DataFrame
            origen = row['Source Airport Code']
            destino = row['Destination Airport Code']
            lat_origen = row['Source Airport Latitude']
            lon_origen = row['Source Airport Longitude']
            lat_destino = row['Destination Airport Latitude']
            lon_destino = row['Destination Airport Longitude']

            distancia = self.find_distance(lat_origen, lon_origen, lat_destino, lon_destino)

            # Registrar la conexión entre el aeropuerto de origen y el aeropuerto de destino con la distancia calculada
            if origen not in grafo:
                grafo[origen] = {destino: distancia}
            else:
                grafo[origen][destino] = distancia

            # Registrar la conexión entre el aeropuerto de destino y el aeropuerto de origen con la misma distancia
            if destino not in grafo:
                grafo[destino] = {origen: distancia}
            else:
                grafo[destino][origen] = distancia

        return grafo

    def extract(self):
        """
        Extaer la información de los aeropuertos del DF. Guarda la información en un diccionario
        """
        informacion = {}
        for _, airport in self.df.iterrows():
            airport_code = airport['Source Airport Code']
            airport_name = airport['Source Airport Name']
            city = airport['Source Airport City']
            country = airport['Source Airport Country']
            latitud = airport['Source Airport Latitude']
            longitud = airport['Source Airport Longitude']

            informacion[airport_code] = {
                'Airport Name': airport_name,
                'City': city,
                'Country': country,
                'Latitude': latitud,
                'Longitude': longitud
            }
        return informacion

    def unique_code(self):
        # Codigos de forma unica
        airport_codes = set(self.df['Source Airport Code']).union(set(self.df['Destination Airport Code']))
        return airport_codes

    def matriz_distance(self):
        """
        Construye la matriz de distancias entre pares de puertos con el DF.
        """
        n = len(self.airport_codes)
        distance_matrix = [[0] * n for _ in range(n)]           # Matriz en 0

        airport_index_map = {code: i for i, code in enumerate(self.airport_codes)}
        # Calculo de distancias y llenado de matriz
        for _, row in self.df.iterrows():
            from_airport = row['Source Airport Code']
            to_airport = row['Destination Airport Code']
            distancia = self.find_distance(row['Source Airport Latitude'], row['Source Airport Longitude'],
                                          row['Destination Airport Latitude'], row['Destination Airport Longitude'])
            from_index = airport_index_map[from_airport]
            to_index = airport_index_map[to_airport]
            distance_matrix[from_index][to_index] = distancia
            distance_matrix[to_index][from_index] = distancia

        return distance_matrix



    # CALCULOS Y ALGORITMOS

    def dijkstra(self,code):
        """
        Implementa el algoritmo de Dijkstra para el camino con la distancia más corta desde un origen dado,
        Args: code(str)     Returns: list
        """
        indice = list(self.airport_codes).index(code)
        V = len(self.distance_matrix)
        dist = [float('inf')] * V
        dist[indice] = 0
        spt_set = [False] * V

        for _ in range(V):
            u = self.min_distance(dist, spt_set)
            spt_set[u] = True
            for v in range(V):
                if self.distance_matrix[u][v] > 0 and not spt_set[v] and dist[v] > dist[u] + self.distance_matrix[u][v]:
                    dist[v] = dist[u] + self.distance_matrix[u][v]
        return dist

    def disjkstra_camino(self, origen, destino):
        """
        Encuentra el camino mínimo entre dos puertos.
        Args: origen(str) destino(srt)      Returns: list
        """
        distancias = {aeropuerto: float('inf') for aeropuerto in self.grafo}
        predecesor = {aeropuerto: None for aeropuerto in self.grafo}

        distancias[origen] = 0
        cola = [(0, origen)]

        while cola:
            distancia_actual, airport_actual = heapq.heappop(cola)
            if airport_actual == destino:
                break
            if distancia_actual > distancias[airport_actual]:
                continue

            for adyacente, distancia in self.grafo[airport_actual].items():
                new_dist = distancia_actual + distancia

                if new_dist < distancias[adyacente]:
                    distancias[adyacente] = new_dist
                    predecesor[adyacente] = airport_actual

                    heapq.heappush(cola, (new_dist, adyacente))

        camino = []
        actual = destino
        while actual is not None:
            camino.append(actual)
            actual = predecesor[actual]
        camino.reverse()

        return camino

    def find_distance(self, lat1, long1, lat2, long2):
        """
        Calculo de la distancia en kilometros entre dos puntos dados por sus coordenadas
        Args: (floats) Latitudes y longitudes   returns: (float) distancia en kilometros
        """
        r = 6378.1  #Radio de la Tierra en kilómetros

        # Coordenadas de grados a radianes
        lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])

        # Calcular diferencias de latitud y longitud
        d_lat = lat2 - lat1
        d_long = long2 - long1

        # Aplicar la fórmula de Haversine
        a = sin(d_lat/2)**2 + cos(lat1) * cos(lat2) * sin(d_long/2)**2
        c = 2 * asin(sqrt(a))

        return r * c

    def min_distance(self, dist, spt_set):
        """
        Encuentra el índice de la distancia mínima en el conjunto de caminos encontrados
        Args: distancias(list) spt_set(list)      returns: indice minimo(int)
        """
        min_val = float('inf')
        min_index = 0

        for v in range(len(dist)):
            if dist[v] < min_val and not spt_set[v]:
                min_val = dist[v]
                min_index = v

        return min_index


    def get_total_distance(self, camino):
        """
        Calcula la distancia total recorrida en un camino dado.
        Args: camino (list)     Returns: distancia total (float)
        """
        dt = 0
        for i in range(len(camino) - 1):
                actual = camino[i]
                next_air = camino[i + 1]
                lat_actual, long_actual = actual
                latitud_next, long_next = next_air
                distance_btw = self.find_distance(lat_actual, long_actual, latitud_next, long_next)
                dt += distance_btw
        return dt



    # OBTENER INFORMACIÓN

    def get_long_lat(self, code):
        """
        Obtiene la latitud y longiud dado un codigo indentificado
        Args: codigo(int)       Returns: latitud (int) longitud (int)
        """
                # Obtener latitud y longitud dado un codigo
        airport_info = self.df[self.df['Source Airport Code'] == code]
        if airport_info.empty:
            print("No se encontró información para el aeropuerto especificado.")
            return None, None
        else:
            return airport_info.iloc[0]['Source Airport Latitude'], airport_info.iloc[0]['Source Airport Longitude']

    def showinfo(self, code):
        """
        Muestra información sobre un aeropuerto en el mapa
        """
        # Busca el aeropuerto en las ubicaciones de origen
        airport_info = self.df[(self.df['Source Airport Code'] == code)]

        if  airport_info.empty:
            #Si no se encuentra en las ubicaciones de origen, buscar en las de destino
            airport_info = self.df[(self.df['Destination Airport Code'] == code)]

        if not  airport_info.empty:
            # Imprimir la información del aeropuerto
            print("Información del Aeropuerto:")
            print(f"Código: {code}")
            print(f"Nombre: {airport_info.iloc[0]['Source Airport Name']}")
            print(f"Ciudad: {airport_info.iloc[0]['Source Airport City']}")
            print(f"País: {airport_info.iloc[0]['Source Airport Country']}")
            latitud_rounded = round(airport_info.iloc[0]['Source Airport Latitude'], 6)
            longitud_rounded = round(airport_info.iloc[0]['Source Airport Longitude'], 6)
            print(f"Latitud: {latitud_rounded}")
            print(f"Longitud: {longitud_rounded}")


            folium.Marker(location=[latitud_rounded, longitud_rounded],
                      popup=f"Aeropuerto: {airport_info.iloc[0]['Source Airport Name']}",
                      icon=folium.Icon(color='blue')).add_to(self.mapa)
            display(self.mapa)
        else:
            print("El aeropuerto con el código especificado no se encuentra en el archivo.")



    # VIZUALIZAR Y PRESENTAR

    def show_puntos(self, airport_org, airport_dest):
        """
        Muestra los puntos de origen y destino en el mapa como marcadores
        """
        self.mapa = folium.Map(location=[0, 0], zoom_start=2)

        org_info = self.airport_info.get(airport_org)
        dest_info = self.airport_info.get(airport_dest)

        if org_info and dest_info:
            # Generar el contenido del popup para cada aeropuerto
            org_popup_content = self.popup(org_info['Airport Name'], airport_org, org_info['City'], org_info['Country'], org_info['Latitude'], org_info['Longitude'])
            dest_popup_content = self.popup(dest_info['Airport Name'], airport_dest, dest_info['City'], dest_info['Country'], dest_info['Latitude'], dest_info['Longitude'])

            # Marcar los aeropuertos en el mapa con su respectiva información en el popup
            self.marcar_airports(org_info['Latitude'], org_info['Longitude'], org_popup_content)
            self.marcar_airports(dest_info['Latitude'], dest_info['Longitude'], dest_popup_content)

            # Mostrar el mapa con los marcadores
            display(self.mapa)
        else:
            print("La información completa de uno o ambos aeropuertos no está disponible.")


    def show_camino(self, camino):
        """
        Muestra el camino mínimo en el mapa como una serie de marcadores y una línea que los conecta.
        """
        mapa = folium.Map(location=[0, 0], zoom_start=2)
        for i in range(len(camino) - 1):

            aeropuerto_actual = camino[i]
            siguiente_aeropuerto = camino[i + 1]
            latitud_actual, longitud_actual = aeropuerto_actual
            latitud_siguiente, longitud_siguiente = siguiente_aeropuerto

            # Calcula la distancia entre los aeropuertos
            distancia_entre_aeropuertos = self.find_distance(latitud_actual, longitud_actual, latitud_siguiente, longitud_siguiente)

            # Construye el contenido del popup HTML
            popup_content = f"<b>Distancia entre aeropuertos:</b> {round(distancia_entre_aeropuertos, 2)} km"

            # Añadir marcadores
            folium.CircleMarker(location=[latitud_actual, longitud_actual],
                                radius=5,
                                color='blue',
                                fill=True,
                                fill_color='blue',
                                fill_opacity=1,
                                popup=f"<br>Latitud: {latitud_actual}<br>Longitud: {longitud_actual}").add_to(mapa)

            folium.CircleMarker(location=[latitud_siguiente, longitud_siguiente],
                                radius=5,
                                color='blue',
                                fill=True,
                                fill_color='blue',
                                fill_opacity=1,
                                popup=f"<br>Latitud: {latitud_siguiente}<br>Longitud: {longitud_siguiente}").add_to(mapa)

            # Añade una línea entre los aeropuertos con la distancia como etiqueta
            linea = folium.PolyLine(locations=[aeropuerto_actual, siguiente_aeropuerto], color='purple', weight=4, popup=popup_content)
            mapa.add_child(linea)

        display(mapa)


    def camino(self, camino):
        """
        Muestra el camino minimo obtenido dado el calculo de las distancias entre coodernadas. Calcula la distancia total recorrida en el camino.
        Args: camino (list)     Returns: None
        """
        coordenadas = []

        for i in range (len(camino) - 1):
            origen = camino[i]
            destino = camino[i + 1]
            lat_origen, long_origen = self.get_long_lat(origen)
            lat_destino, long_destino = self.get_long_lat(destino)
            coordenadas.append((lat_origen, long_origen))
            coordenadas.append((lat_destino, long_destino))

        self.show_camino(coordenadas)

        distancia_total = self.get_total_distance(coordenadas)
        print(f" \n La distancia total recorrida es: {round(distancia_total, 2)} km")

    def caminos_largos(self, code):
        """
        Calcula con el algoritmo Dijkstra, los 10 caminos minimos más largos, imprime en consola la información de cada uno de los obtenidos.
        """
        dist = self.dijkstra(code)
        print("Los 10 caminos más largos son:")
        vertices = []

        for node, distance in enumerate(dist):
            if distance != float('inf'):
                vertices.append((node, distance))

        vertices.sort(key=lambda x: x[1], reverse=True)

        for i in range(min(10, len(vertices))):
            airport_code = list(self.airport_codes)[vertices[i][0]]
            airport_data = self.airport_info.get(airport_code)
            if airport_data:
                print(f"{i+1}. Aeropuerto:")
                print("Código:", airport_code)
                print("Nombre:", airport_data['Airport Name'])
                print("Ciudad:", airport_data['City'])
                print("País:", airport_data['Country'])
                print("Latitud:", round(airport_data['Latitude'], 6))
                print("Longitud:", round(airport_data['Longitude'], 6))
                print("Distancia:", round(vertices[i][1], 3), "km")
                print()
            else:
                print(f"No se encontró información para el aeropuerto con código {airport_code}")




    #AGREGAR TODOS LOS AIRPORTS
    def popup(self, name, code, ciudad, pais, latitud, longitud):
        popup_content = f"Nombre : {name}\n"
        popup_content += f"Código: {code}\n"
        popup_content += f"Ciudad: {ciudad}\n"
        popup_content += f"País: {pais}\n"
        popup_content += f"Latitud: {latitud}\n"
        popup_content += f"Longitud: {longitud}\n"
        return popup_content

    def marcar_airports(self, latitud, longitud, popup_content):
        folium.CircleMarker(location=[latitud, longitud],
                            radius=3,
                            color='blue',
                            fill=True,
                            fill_color='blue',
                            fill_opacity=0.7,
                            popup=folium.Popup(popup_content, parse_html=True)).add_to(self.mapa)

    def agregate_airports(self):
        """
        Agrega marcadores de puertos al mpa y los muestra
        """
        self.ubicaciones_agregadas = set()
        # Control de agregados

        for index, row in self.df.iterrows():
            code = row['Source Airport Code']
            latitud = row['Source Airport Latitude']
            longitud = row['Source Airport Longitude']
            name = row['Source Airport Name']
            ciudad = row['Source Airport City']
            pais = row['Source Airport Country']

            # Verifica si el aeropuerto ya está agregado
            if (latitud, longitud) in self.ubicaciones_agregadas:
                continue

            # Construye el contenido del popup HTML
            popup_content =  self.popup(name, code, ciudad, pais, latitud, longitud)
            # Añade un circlemarker para el aeropuerto con el popup personalizado
            self.marcar_airports(latitud, longitud, popup_content)

            # Agrega el aeropuerto al conjunto de ubicaciones agregadas
            self.ubicaciones_agregadas.add((latitud, longitud))


*MAIN*

In [ ]:
#_________________________________
def ln ():
    print("_____________________________________")

def main():
    map_handler = MapHandler()

    while True:
        ln()
        print("MANEJO DE AEROPUERTOS")
        ln()
        print("1. INFORMACIÓN DEL AEROPUERTO")
        print("2. CAMINO MÍNIMO ENTRE AERPUERTOS")
        print("3. EXIT")
        ln()

        opcion = input("Ingrese su opción: ")

        if opcion == '1':
            ln()
            # Solicitar codigo a buscar
            codigo = input("Ingrese el código del aeropuerto: ").upper()
            ln()
            map_handler.showinfo(codigo)        # Mostrar información y ubicación.
            ln()
            map_handler.caminos_largos(codigo)  # Imprime en pantalla los 10 mas largos.

        elif opcion == '2':
            ln()
            # Solicitar codigos de origen y destino
            origen = input("Código aeropuerto de origen: ").upper()
            destino = input("Código aeropuerto de destino: ").upper()
            ln()
            print("Información de los aereopuertos")
            ln()
            map_handler.show_puntos(origen, destino)    # Mostrar puntos e información en el mapa
            ln()
            camino_minimo = map_handler.disjkstra_camino(origen, destino)   # Calcular camino
            print(f" \n El camino mínimo desde {origen} hasta {destino} es: {camino_minimo} \n ")
            ln()
            map_handler.camino(camino_minimo)           # Muestra el camino en mapa
            ln()
        elif opcion == '3':
            ln()
            print("Que la fuerza te acompañe")
            break
        else:
            print("Opción no válida. Por favor, ingrese una opción válida.")

if __name__ == "__main__":
    main()